# Economic Forecasting with LSTM: GDP and Inflation Analysis

**Duration:** 60-90 minutes  
**Goal:** Train LSTM models to forecast GDP growth and inflation using historical macroeconomic data

## What You'll Learn

- Load and explore economic time series data from FRED (Federal Reserve Economic Data)
- Perform stationarity testing and preprocessing
- Build LSTM neural networks for economic forecasting
- Evaluate forecast accuracy and generate predictions
- Understand business cycle dynamics

## Dataset

We'll use **FRED** datasets:
- GDP growth (quarterly, 1947-2024)
- CPI inflation (monthly, 1947-2024)
- Unemployment rate (monthly, 1948-2024)
- Federal funds rate (monthly, 1954-2024)
- Source: Federal Reserve Bank of St. Louis

No API key needed - let's get started!

## 1. Setup and Data Loading

In [ ]:
# Import libraries (all pre-installed in Colab/Studio Lab)
import warnings
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings("ignore")

# Statistical libraries
# Deep learning
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.stattools import adfuller
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (14, 6)
plt.rcParams["font.size"] = 11

print("✓ Libraries loaded successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Analysis date: {datetime.now().strftime('%Y-%m-%d')}")

In [ ]:
# Load economic data from FRED (Federal Reserve Economic Data)
# Using pandas-datareader to fetch data directly

try:
    import pandas_datareader as pdr
except ImportError:
    print("Installing pandas-datareader...")
    !pip install pandas-datareader -q
    import pandas_datareader as pdr

# Set date range
start_date = "1990-01-01"
end_date = "2024-01-01"

print("Downloading economic data from FRED...")
print("This may take 2-3 minutes...\n")

# GDP (Quarterly, Real GDP)
gdp = pdr.DataReader("GDPC1", "fred", start_date, end_date)
print(f"✓ GDP data: {len(gdp)} quarters")

# CPI (Monthly, Consumer Price Index)
cpi = pdr.DataReader("CPIAUCSL", "fred", start_date, end_date)
print(f"✓ CPI data: {len(cpi)} months")

# Unemployment Rate (Monthly)
unemployment = pdr.DataReader("UNRATE", "fred", start_date, end_date)
print(f"✓ Unemployment data: {len(unemployment)} months")

# Federal Funds Rate (Monthly)
fed_funds = pdr.DataReader("FEDFUNDS", "fred", start_date, end_date)
print(f"✓ Federal Funds Rate data: {len(fed_funds)} months")

print("\n✓ All data downloaded successfully!")

### Understanding Economic Indicators

**GDP (Gross Domestic Product):** Total value of goods and services produced
- Measures overall economic activity
- Reported quarterly
- Growth rate indicates expansion or contraction

**CPI (Consumer Price Index):** Average price level of consumer goods
- Measures inflation
- Year-over-year change = inflation rate
- Central banks target ~2% inflation

**Unemployment Rate:** Percentage of labor force without jobs
- Key labor market indicator
- Counter-cyclical (rises in recessions)

**Federal Funds Rate:** Interest rate banks charge each other
- Set by Federal Reserve (monetary policy)
- Influences borrowing costs economy-wide

## 2. Data Preprocessing

In [ ]:
# Calculate GDP growth rate (quarter-over-quarter percentage change)
gdp_growth = gdp.pct_change(4) * 100  # Year-over-year growth
gdp_growth.columns = ["GDP_Growth"]
gdp_growth = gdp_growth.dropna()

# Calculate inflation rate (year-over-year CPI change)
inflation = cpi.pct_change(12) * 100  # Year-over-year inflation
inflation.columns = ["Inflation"]
inflation = inflation.dropna()

# Rename columns for clarity
unemployment.columns = ["Unemployment"]
fed_funds.columns = ["Fed_Funds_Rate"]

# Resample GDP (quarterly) to monthly using forward fill
gdp_growth_monthly = gdp_growth.resample("MS").ffill()

# Merge all indicators
economic_data = pd.concat([gdp_growth_monthly, inflation, unemployment, fed_funds], axis=1).dropna()

print(
    f"✓ Processed data: {len(economic_data)} months ({economic_data.index[0].year}-{economic_data.index[-1].year})"
)
print(f"\nData shape: {economic_data.shape}")
print("\nFirst few rows:")
economic_data.head()

In [ ]:
# Summary statistics
print("=== Economic Indicators Summary Statistics ===")
print(economic_data.describe())

# Correlations
print("\n=== Correlation Matrix ===")
print(economic_data.corr())

In [ ]:
# Visualize all indicators
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# GDP Growth
axes[0].plot(economic_data.index, economic_data["GDP_Growth"], color="steelblue", linewidth=1.5)
axes[0].axhline(y=0, color="red", linestyle="--", alpha=0.5)
axes[0].fill_between(
    economic_data.index,
    0,
    economic_data["GDP_Growth"],
    where=economic_data["GDP_Growth"] < 0,
    color="red",
    alpha=0.2,
    label="Recessions",
)
axes[0].set_title("GDP Growth Rate (Year-over-Year %)", fontsize=12, fontweight="bold")
axes[0].set_ylabel("Growth %")
axes[0].grid(True, alpha=0.3)

# Inflation
axes[1].plot(economic_data.index, economic_data["Inflation"], color="orange", linewidth=1.5)
axes[1].axhline(y=2, color="green", linestyle="--", alpha=0.5, label="Fed Target (2%)")
axes[1].set_title("Inflation Rate (Year-over-Year CPI %)", fontsize=12, fontweight="bold")
axes[1].set_ylabel("Inflation %")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Unemployment
axes[2].plot(economic_data.index, economic_data["Unemployment"], color="red", linewidth=1.5)
axes[2].set_title("Unemployment Rate", fontsize=12, fontweight="bold")
axes[2].set_ylabel("Unemployment %")
axes[2].grid(True, alpha=0.3)

# Federal Funds Rate
axes[3].plot(economic_data.index, economic_data["Fed_Funds_Rate"], color="purple", linewidth=1.5)
axes[3].set_title("Federal Funds Rate", fontsize=12, fontweight="bold")
axes[3].set_ylabel("Rate %")
axes[3].set_xlabel("Date")
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Visualization shows business cycles, monetary policy changes, and economic shocks")

## 3. Stationarity Testing

In [ ]:
# Augmented Dickey-Fuller test for stationarity
def test_stationarity(series, name):
    """
    Perform Augmented Dickey-Fuller test for stationarity.
    Null hypothesis: series has a unit root (non-stationary)
    """
    result = adfuller(series.dropna())
    print(f"\n=== Stationarity Test: {name} ===")
    print(f"ADF Statistic: {result[0]:.4f}")
    print(f"P-value: {result[1]:.4f}")
    print("Critical Values:")
    for key, value in result[4].items():
        print(f"  {key}: {value:.4f}")

    if result[1] < 0.05:
        print(f"✓ {name} is STATIONARY (p < 0.05)")
        return True
    else:
        print(f"✗ {name} is NON-STATIONARY (p >= 0.05) - may need differencing")
        return False


# Test each indicator
for col in economic_data.columns:
    test_stationarity(economic_data[col], col)

## 4. LSTM Model for GDP Growth Forecasting

In [ ]:
# Prepare data for LSTM
def create_sequences(data, lookback=12, forecast_horizon=3):
    """
    Create sequences for LSTM training.

    Args:
        data: Time series data
        lookback: Number of past time steps to use as input
        forecast_horizon: Number of future time steps to predict

    Returns:
        X: Input sequences, y: Target values
    """
    X, y = [], []
    for i in range(lookback, len(data) - forecast_horizon + 1):
        X.append(data[i - lookback : i])
        y.append(data[i + forecast_horizon - 1])  # Predict 'forecast_horizon' steps ahead
    return np.array(X), np.array(y)


# Focus on GDP growth forecasting
target_variable = "GDP_Growth"
data = economic_data[[target_variable]].values

# Normalize data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Split into train/test (80/20)
train_size = int(len(data_scaled) * 0.8)
train_data = data_scaled[:train_size]
test_data = data_scaled[train_size:]

# Create sequences
lookback = 12  # Use 12 months of history
forecast_horizon = 3  # Forecast 3 months ahead

X_train, y_train = create_sequences(train_data, lookback, forecast_horizon)
X_test, y_test = create_sequences(test_data, lookback, forecast_horizon)

print(f"✓ Training data: {X_train.shape[0]} sequences")
print(f"  Input shape: {X_train.shape}")
print(f"  Output shape: {y_train.shape}")
print(f"\n✓ Test data: {X_test.shape[0]} sequences")
print(f"  Forecast horizon: {forecast_horizon} months")

In [ ]:
# Build LSTM model
print("Building LSTM model...\n")

model = Sequential(
    [
        LSTM(64, activation="relu", return_sequences=True, input_shape=(lookback, 1)),
        Dropout(0.2),
        LSTM(32, activation="relu", return_sequences=False),
        Dropout(0.2),
        Dense(16, activation="relu"),
        Dense(1),
    ]
)

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

print(model.summary())

In [ ]:
# Train model
print("Training LSTM model...")
print("This will take 60-75 minutes on GPU, longer on CPU\n")

# Early stopping callback
early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Train
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1,
)

print("\n✓ Training complete!")

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history.history["loss"], label="Training Loss", linewidth=2)
axes[0].plot(history.history["val_loss"], label="Validation Loss", linewidth=2)
axes[0].set_title("Model Loss During Training", fontsize=12, fontweight="bold")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss (MSE)")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE
axes[1].plot(history.history["mae"], label="Training MAE", linewidth=2)
axes[1].plot(history.history["val_mae"], label="Validation MAE", linewidth=2)
axes[1].set_title("Model MAE During Training", fontsize=12, fontweight="bold")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("MAE")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Model Evaluation

In [ ]:
# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Inverse transform to get actual values
y_train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))
y_train_pred_actual = scaler.inverse_transform(y_train_pred)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
y_test_pred_actual = scaler.inverse_transform(y_test_pred)

# Calculate metrics
train_rmse = np.sqrt(mean_squared_error(y_train_actual, y_train_pred_actual))
train_mae = mean_absolute_error(y_train_actual, y_train_pred_actual)
test_rmse = np.sqrt(mean_squared_error(y_test_actual, y_test_pred_actual))
test_mae = mean_absolute_error(y_test_actual, y_test_pred_actual)

print("=== Model Performance ===")
print("\nTraining Set:")
print(f"  RMSE: {train_rmse:.4f}")
print(f"  MAE:  {train_mae:.4f}")
print("\nTest Set:")
print(f"  RMSE: {test_rmse:.4f}")
print(f"  MAE:  {test_mae:.4f}")

# Directional accuracy (did we predict the right direction?)
test_directions_actual = np.diff(y_test_actual.flatten()) > 0
test_directions_pred = np.diff(y_test_pred_actual.flatten()) > 0
directional_accuracy = np.mean(test_directions_actual == test_directions_pred) * 100

print(f"\nDirectional Accuracy: {directional_accuracy:.1f}%")
print("(Percentage of time we correctly predicted whether GDP growth would increase or decrease)")

In [ ]:
# Visualize predictions vs actual
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Training set
axes[0].plot(y_train_actual, label="Actual", linewidth=2, alpha=0.7)
axes[0].plot(y_train_pred_actual, label="Predicted", linewidth=2, alpha=0.7)
axes[0].set_title(
    f"GDP Growth Forecast - Training Set (RMSE: {train_rmse:.3f})", fontsize=12, fontweight="bold"
)
axes[0].set_ylabel("GDP Growth %")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Test set
axes[1].plot(y_test_actual, label="Actual", linewidth=2, alpha=0.7, color="steelblue")
axes[1].plot(y_test_pred_actual, label="Predicted", linewidth=2, alpha=0.7, color="orange")
axes[1].set_title(
    f"GDP Growth Forecast - Test Set (RMSE: {test_rmse:.3f})", fontsize=12, fontweight="bold"
)
axes[1].set_ylabel("GDP Growth %")
axes[1].set_xlabel("Time Step")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📈 Model captures GDP growth trends with reasonable accuracy")

In [ ]:
# Residual analysis
test_residuals = y_test_actual.flatten() - y_test_pred_actual.flatten()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Residual plot
axes[0].scatter(y_test_pred_actual, test_residuals, alpha=0.6)
axes[0].axhline(y=0, color="red", linestyle="--")
axes[0].set_title("Residual Plot", fontsize=12, fontweight="bold")
axes[0].set_xlabel("Predicted GDP Growth")
axes[0].set_ylabel("Residuals")
axes[0].grid(True, alpha=0.3)

# Residual distribution
axes[1].hist(test_residuals, bins=20, edgecolor="black", alpha=0.7)
axes[1].set_title("Residual Distribution", fontsize=12, fontweight="bold")
axes[1].set_xlabel("Residual")
axes[1].set_ylabel("Frequency")
axes[1].grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

print("Residual statistics:")
print(f"  Mean: {np.mean(test_residuals):.4f}")
print(f"  Std: {np.std(test_residuals):.4f}")

## 6. Generate Future Forecasts

In [ ]:
# Generate multi-step ahead forecasts
def generate_forecast(model, last_sequence, scaler, n_steps=12):
    """
    Generate multi-step ahead forecast using recursive strategy.
    """
    forecast = []
    current_sequence = last_sequence.copy()

    for _ in range(n_steps):
        # Predict next step
        next_pred = model.predict(current_sequence.reshape(1, lookback, 1), verbose=0)
        forecast.append(next_pred[0, 0])

        # Update sequence (roll forward)
        current_sequence = np.roll(current_sequence, -1)
        current_sequence[-1] = next_pred[0, 0]

    # Inverse transform
    forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    return forecast.flatten()


# Get last sequence from test data
last_sequence = test_data[-lookback:]

# Generate 12-month forecast
forecast_steps = 12
future_forecast = generate_forecast(model, last_sequence, scaler, forecast_steps)

# Create forecast dates
last_date = economic_data.index[-1]
forecast_dates = pd.date_range(
    start=last_date + pd.DateOffset(months=1), periods=forecast_steps, freq="MS"
)

print("=== 12-Month GDP Growth Forecast ===")
for date, value in zip(forecast_dates, future_forecast):
    print(f"{date.strftime('%Y-%m')}: {value:.2f}%")

In [ ]:
# Visualize forecast
fig, ax = plt.subplots(figsize=(14, 6))

# Historical data (last 5 years)
historical = economic_data[target_variable][-60:]
ax.plot(historical.index, historical.values, label="Historical", linewidth=2, color="steelblue")

# Forecast
ax.plot(
    forecast_dates,
    future_forecast,
    label="Forecast",
    linewidth=2,
    color="orange",
    linestyle="--",
    marker="o",
)

# Uncertainty bands (simple: +/- 1 std of test errors)
uncertainty = np.std(test_residuals)
ax.fill_between(
    forecast_dates,
    future_forecast - uncertainty,
    future_forecast + uncertainty,
    alpha=0.2,
    color="orange",
    label="Uncertainty Band",
)

ax.axhline(y=0, color="red", linestyle="--", alpha=0.5)
ax.set_title("GDP Growth: Historical Data and 12-Month Forecast", fontsize=14, fontweight="bold")
ax.set_xlabel("Date", fontsize=12)
ax.set_ylabel("GDP Growth (%)", fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

avg_forecast = np.mean(future_forecast)
print(f"\n📊 Average forecasted GDP growth (next 12 months): {avg_forecast:.2f}%")
outlook = ("Outlook: Strong economic growth expected" if avg_forecast > 2
           else "Outlook: Moderate economic growth expected" if avg_forecast > 0
           else "Outlook: Economic contraction expected (recession risk)")
print(f"   {outlook}")

## 7. Key Findings Summary

In [ ]:
# Generate summary report
print("=" * 70)
print("ECONOMIC FORECASTING SUMMARY")
print("=" * 70)
print(
    f"\n📅 Data Period: {economic_data.index[0].year} to {economic_data.index[-1].year} ({len(economic_data)} months)"
)
print("\n🎯 TARGET VARIABLE: GDP Growth (Year-over-Year %)")
print(f"   • Historical average: {economic_data['GDP_Growth'].mean():.2f}%")
print(f"   • Historical std dev: {economic_data['GDP_Growth'].std():.2f}%")
print(
    f"   • Min growth: {economic_data['GDP_Growth'].min():.2f}% ({economic_data['GDP_Growth'].idxmin().year})"
)
print(
    f"   • Max growth: {economic_data['GDP_Growth'].max():.2f}% ({economic_data['GDP_Growth'].idxmax().year})"
)
print("\n🤖 MODEL ARCHITECTURE:")
print("   • Type: LSTM Neural Network")
print(f"   • Lookback window: {lookback} months")
print(f"   • Forecast horizon: {forecast_horizon} months ahead")
print(f"   • Training samples: {len(X_train)}")
print(f"   • Test samples: {len(X_test)}")
print("\n📊 MODEL PERFORMANCE:")
print(f"   • Test RMSE: {test_rmse:.4f}")
print(f"   • Test MAE: {test_mae:.4f}")
print(f"   • Directional accuracy: {directional_accuracy:.1f}%")
print("\n🔮 12-MONTH FORECAST:")
print(f"   • Average GDP growth: {avg_forecast:.2f}%")
print(f"   • Range: {future_forecast.min():.2f}% to {future_forecast.max():.2f}%")
print(f"   • Uncertainty (±1 std): ±{uncertainty:.2f}%")
print("\n✅ CONCLUSION:")
print("   The LSTM model successfully captures GDP growth patterns and provides")
print(f"   reasonable forecasts with {directional_accuracy:.0f}% directional accuracy.")
print(f"   Model performance on test data (RMSE: {test_rmse:.3f}) is acceptable")
print("   for short-term economic forecasting applications.")
print("=" * 70)

## 🎓 What You Learned

In just 60-90 minutes, you:

1. ✅ Downloaded and processed economic time series from FRED
2. ✅ Performed stationarity testing (ADF test)
3. ✅ Built and trained LSTM neural network for forecasting
4. ✅ Evaluated model performance (RMSE, MAE, directional accuracy)
5. ✅ Generated 12-month GDP growth forecasts
6. ✅ Understood business cycle dynamics and economic indicators

## 🚀 Next Steps

### Ready for More?

**Tier 1: SageMaker Studio Lab (4-8 hours, free)**
- Multi-country panel data analysis (10GB dataset)
- Ensemble forecasting with 5-6 models (ARIMA, VAR, LSTM, Prophet, XGBoost)
- Cross-country economic spillover analysis
- Persistent storage and model checkpoints
- Granger causality testing

**Tier 2: AWS Starter (4-6 hours, $5-15)**
- Store economic data in S3
- Automated data pipelines with Lambda
- SageMaker training jobs with hyperparameter tuning
- Real-time indicator updates

**Tier 3: Production Infrastructure (1-2 weeks, $50-500/month)**
- 100+ countries and 500+ indicators
- Distributed training with SageMaker
- API for forecast delivery
- Automated retraining pipelines
- Real-time monitoring and alerts

## 📚 Learn More

- **Data Source:** [FRED - Federal Reserve Economic Data](https://fred.stlouisfed.org/)
- **Research:** [NBER Working Papers](https://www.nber.org/papers)
- **Forecasting Methods:** [Hyndman & Athanasopoulos - Forecasting: Principles and Practice](https://otexts.com/fpp3/)
- **LSTM for Economics:** [Deep Learning for Economic Forecasting (2023)](https://arxiv.org/)

---

**Generated with [Claude Code](https://claude.com/claude-code)**